<a href="https://colab.research.google.com/github/gvanathip/Thai-Travel-Open-AI-Chatbot-with-GPT2/blob/main/scrap_attraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'https://thai.tourismthailand.org/Search-result/attraction?sort_by=datetime_updated_desc&page=1&perpage=15&menu=attraction'

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
driver.get(url)

In [ ]:
import time

In [ ]:
p = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div[2]/div/div/div[3]/div/div').get_attribute("innerText")
time.sleep(5)
p

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
links = []
plcs = dict()
for i in range(int(p.split('\n')[-1])):
  urlx = url.replace('&page=1&','&page='+str(i+1)+'&')
  #print(urlx)

  driver.get(urlx)
  time.sleep(5)
  text = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div[2]/div/div/div[2]/div/div').get_attribute("innerHTML")
  # print(text)
  for i in text.split('><'):
    if 'href' in str(i):
      link = 'https://thai.tourismthailand.org/Accessibility'+i.split('"')[1]
      links.append(link)
      print('https://thai.tourismthailand.org/Accessibility'+i.split('"')[1])

      plc = text.split('<div class="">')[1].split('</div>')[0]
      plcs[plc] = link
  
print(plcs)
plcs_ = pd.DataFrame.from_dict(plcs, orient='index')
print(plcs_)
plcs_.to_csv('/content/drive/MyDrive/IS-DS/'+'attraction_list.csv')
links = list(set(links))
print(links)

In [ ]:
plcs_

In [ ]:
temp = pd.read_csv(('/content/drive/MyDrive/IS-DS/'+'attraction_list.csv'))

In [ ]:
links = list(temp['0'])

In [ ]:
trip = pd.read_csv('/content/drive/MyDrive/IS-DS/trip_planners.csv')
trip.head()

In [ ]:
for i in list(trip.place_link):
  for j in i[1:-1].split(','):
    try:
      print('https:'+j.split(':')[2])
      links.append('https:'+j.split(':')[2])
    except:
      continue

In [ ]:
links = list(set(links))

In [ ]:
attractions = []
categories = []
provinces = []
pictures = []
types = []
addresses = []
numbers = []
informations = []
importants = []
maps = []
lnk = []

for i in list(set(links)):

  print(i)
  lnk.append(i)
  urlxx = requests.get(i)
  soup = BeautifulSoup(urlxx.content, "html.parser")

  driver.get(i)
  time.sleep(3)
  
  text = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[1]/div[2]/div').get_attribute("innerText")

  attractions.append(text.replace('\xa0','').split('\n')[0])

  try:
    category = text.replace('\xa0','').split('\n')[1].split('|')[0]
    categories.append(category)
  except:
    categories.append('')

  try:
    province = text.replace('\xa0','').split('\n')[1].split('|')[1]
    provinces.append(province)
  except:
    provinces.append('')

  try:
    picture = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[1]/div[4]/div[1]/picture/img').get_attribute("src")
    pictures.append(picture)
  except:
    pictures.append('')
  
  try:
    type_ = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[1]/div[4]/div[2]/div/p[2]').get_attribute("innerText")
    types.append(type_)
  except:
    types.append('')
  
  try:
    address = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[4]/div[2]/div/div/div[2]/div/div[2]/p[1]').get_attribute("innerText")
    addresses.append(address)
  except:
    addresses.append('')

  try:
    number = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[1]/div[4]/div[2]/div/div/ul/li/a').get_attribute("innerText")
    numbers.append(number)
  except:
    numbers.append('')

  try:
    
    information = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[3]/div/div[2]/div[2]/div/div[2]/div/div/div').get_attribute("innerText")
    information.replace('\xa0','').replace('\n',' ')
    informations.append(information)
  except:
    informations.append('')

  try:
    important = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[3]/div/div[2]/div[5]/div/div/div[2]').get_attribute("innerText")
    important.replace('\xa0','').replace('\n',' ')
    importants.append(important)
  except:
    importants.append('')

  try:
    map_ = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[4]/div[2]/div/div/div[2]/div/div[1]/div/iframe').get_attribute("src")
    maps.append(map_.split('&')[0])
  except:
    maps.append('')


df = pd.DataFrame(list(zip(attractions,
                           categories,
                           provinces,
                           pictures,
                           types,
                           addresses,
                           numbers,
                           informations,
                           importants,
                           maps,
                           lnk)), columns=['place',
                                           'category',
                                           'province',
                                           'picture',
                                           'type',
                                           'address',
                                           'numbers',
                                           'information',
                                           'importants',
                                           'maps',
                                           'link'])
df.to_csv('/content/drive/MyDrive/IS-DS/'+'attraction.csv',index=False,index_label=False)

print(df)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IS-DS/'+'attraction.csv')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df[df.columns] = df.apply(lambda x: x.str.strip())

In [ ]:
import datetime
import numpy as np

In [ ]:
mask = df.category.astype(str).str[0].str.isdigit()
df.loc[mask, 'category'] = np.NaN
cat = list(set(df[df.province.notnull()].category))
print(cat)
df.loc[(df.category.notnull())&(~df.category.isin(cat))&(df.province.isnull()), 'province'] = df['category']
df.loc[df.category==df.province, 'category'] = np.NaN
df = df[df.province.notnull()].drop_duplicates()

In [ ]:
df[df['place']=='เกาะไข่']

In [ ]:
import pandas as pd
trip = pd.read_csv('/content/drive/MyDrive/IS-DS/trip_planners.csv')
trip.head()

In [ ]:
places = []
links = []

for i in list(trip.place_link):
  for j in i[1:-1].split(','):
    try:
      print(j.split(':')[0].replace("'",'').strip())
      places.append(j.split(':')[0].replace("'",'').strip())
      print('https:'+j.split(':')[2].replace("'",'').strip())
      links.append('https:'+j.split(':')[2].replace("'",'').strip())
    except:
      continue

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

attractions = []
categories = []
provinces = []
pictures = []
types = []
addresses = []
numbers = []
informations = []
importants = []
maps = []
lnk = []

for i in list(set(links)):

  print(i)
  lnk.append(i)
  urlxx = requests.get(i)
  soup = BeautifulSoup(urlxx.content, "html.parser")

  driver.get(i)
  time.sleep(3)
  
  text = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[1]/div[2]/div').get_attribute("innerText")
  print('attraction:',text.replace('\xa0','').split('\n')[0])

  attractions.append(text.replace('\xa0','').split('\n')[0])

  try:
    category = text.replace('\xa0','').split('\n')[1].split('|')[0]
    print('category:',category)
    categories.append(category)
  except:
    categories.append('')

  try:
    province = text.replace('\xa0','').split('\n')[1].split('|')[1]
    print('province:',province)
    provinces.append(province)
  except:
    provinces.append('')

  try:
    picture = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[1]/div[4]/div[1]/picture/img').get_attribute("src")
    print('picture:',picture)
    pictures.append(picture)
  except:
    pictures.append('')
  
  try:
    type_ = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[1]/div[4]/div[2]/div/p[2]').get_attribute("innerText")
    print('type:',type_)
    types.append(type_)
  except:
    types.append('')
  
  try:
    address = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[4]/div[2]/div/div/div[2]/div/div[2]/p[1]').get_attribute("innerText")
    print('address:',address)
    addresses.append(address)
  except:
    addresses.append('')

  try:
    number = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[1]/div[4]/div[2]/div/div/ul/li/a').get_attribute("innerText")
    print('number:',number)
    numbers.append(number)
  except:
    numbers.append('')

  try:
    
    information = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[3]/div/div[2]/div[2]/div/div[2]/div/div/div').get_attribute("innerText")
    information.replace('\xa0','').replace('\n',' ')
    print('information:\n',information)
    informations.append(information)
  except:
    informations.append('')

  try:
    important = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[3]/div/div[2]/div[5]/div/div/div[2]').get_attribute("innerText")
    important.replace('\xa0','').replace('\n',' ')
    print('important:\n',important)
    importants.append(important)
  except:
    importants.append('')

  try:
    map_ = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div[4]/div[2]/div/div/div[2]/div/div[1]/div/iframe').get_attribute("src")
    print('map:',map)
    maps.append(map_.split('&')[0])
  except:
    maps.append('')


df = pd.DataFrame(list(zip(attractions,
                           categories,
                           provinces,
                           pictures,
                           types,
                           addresses,
                           numbers,
                           informations,
                           importants,
                           maps,
                           lnk)), columns=['place',
                                           'category',
                                           'province',
                                           'picture',
                                           'type',
                                           'address',
                                           'numbers',
                                           'information',
                                           'importants',
                                           'maps',
                                           'link'])

In [ ]:
df.to_csv('/content/drive/MyDrive/IS-DS/'+'attraction2.csv',index=False,index_label=False)

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/IS-DS/'+'attraction2.csv')

In [ ]:
import numpy as np
mask = df1.category.astype(str).str[0].str.isdigit()
df1.loc[mask, 'category'] = np.NaN
#cat = list(set(df1[df1.province.notnull()].category))
print(cat)
df1.loc[(df1.category.notnull())&(~df1.category.isin(cat))&(df1.province.isnull()), 'province'] = df1['category']
df1.loc[df1.category==df1.province, 'category'] = np.NaN
df1 = df1[df1.province.notnull()].drop_duplicates()

In [ ]:
df

In [ ]:
df1

In [ ]:
dfx = pd.concat([df,df1])

In [ ]:
dfx

In [ ]:
dfx[dfx['place']=='เกาะไข่']

In [ ]:
dfx.to_csv('/content/drive/MyDrive/IS-DS/'+'attraction.csv',index=False,index_label=False)